In [1]:
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from paddle.vision.datasets import MNIST

train_dataset = MNIST(mode='train')

train_data_0 = np.array(train_dataset[0][0])
train_label_0 = np.array(train_dataset[0][1])

plt.figure("Image")
plt.figure(figsize=(3, 3))
plt.imshow(train_data_0, cmap=plt.cm.binary)
plt.title('image')
plt.show()

print('Image data shape:', train_data_0.shape)
print('Image label:', train_label_0)

class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()
        self.fc = paddle.nn.Linear(in_features=784, out_features=1)

    def forward(self, inputs):
        outputs = self.fc(inputs)
        return outputs

model = MNIST()

def train(model):
    model.train()
    train_loader = paddle.io.DataLoader(MNIST(mode='train'), batch_size=16, shuffle=True)
    opt = paddle.optimizer.SGD(learning_rate=0.001, parameters=model.parameters)

    EPOCH_NUM = 10
    lost_list = []
    for epoch in range(EPOCH_NUM):
        for Batch_id, data in enumerate(train_loader()):
            images, labels = data[0], data[1]
            images = norm_img(images).astype('float32')
            labels = labels.astype('float32')

            predicts = model(images)

            loss = F.square_error_cost(predicts, labels)
            avg_loss = paddle.mean(loss)

            if (Batch_id % 600 == 0):
                loss = avg_loss.numpy()[0]
                lost_list.append(loss)
                print('epoch_id:{}, batch_id:{}, loss is :{}', format(epoch, Batch_id, loss))

            avg_loss.backward()
            opt.step()
            opt.clear_grad()

    return lost_list

lost_list = train(model)
